#### Aghsi company
#### noreddine.damane94@gmail.com

# Feature Extraction

## import required libraries

In [1]:
import numpy as np
import math
import cv2 as cv
import matplotlib.pyplot as plt
import pandas as pd
import os

In [2]:
## polynômes radiaux de Zernike
def polyradial(r,n,m):
    Rnm = np.zeros(r.shape)
    for s in range(0, ((n-abs(m))//2)+1):
        c = (-1)**(s) * ( math.factorial(n-s) / ((math.factorial(s)) * (math.factorial((n+abs(m))/2-s)) * (math.factorial((n-abs(m))/2-s))))
        Rnm = Rnm + c *  r**(n - 2*s)
        #print("s=",s)
    return Rnm
    

## zernike moments
def Zernikmoment(img,n,m):
    N  = img.shape[0]
    x = np.arange(0 , N)
    x = np.array([x])
    y = x
    X, Y = np.meshgrid(x, y)
    R = np.sqrt((2*X-N+1)**2+(N-1-2*Y)**2) / N
    Theta = np.arctan2((N-1-2*Y),(2*X-N+1))
    R[R>1]= 0 # r inside unit circle
    Rnm = polyradial(R,n,m)
    Product = img*Rnm*np.exp(-1j*m*Theta)
    #Z = (Product).sum()
    Z = (Product.ravel()).sum()
    cnt = np.count_nonzero(R) # count the number of pixels inside the unit circle
    Z = (n+1)*Z/cnt # normalize the amplitude of moments
    A = abs(Z)
    Phi = np.angle(Z)*180/math.pi
    return [A, round(Phi,2)]

## Main()

### Compute the Zernike moments for an order n = 5 and a repetition m = 1

In [3]:
classes = {
	"circle"   : 1,
	"square"   : 2,
	"star"     : 3,
	"triangle" : 4
}

print('les moments zernike pour n= 5 et m= 1')
n = 5 ; m = 1
features = []
img_path = "archive/shapes/"
for c, idx in classes.items():
    class_folder = img_path + "{}/".format(c)
    for f in os.listdir(class_folder):
        fpath = class_folder + f
        sample = int(f.replace(".png", ""))
        img = cv.imread(fpath, cv.IMREAD_GRAYSCALE)
        img_ = np.logical_not(img)
        img_ = ~img_
        Z_m = Zernikmoment(img_,n,m)
        Z_m = np.array(Z_m)
        Z_m = [idx] + Z_m.reshape((1, 2)).tolist()[0]
        features.append(Z_m)
cols = ["class", "Amplitude", "Phase"]
df = pd.DataFrame(features, columns=cols)
df


les moments zernike pour n= 5 et m= 1


C:\Users\HP\AppData\Local\Temp/ipykernel_7256/3140508661.py:5: DeprecationWarning: Using factorial() with floats is deprecated
  c = (-1)**(s) * ( math.factorial(n-s) / ((math.factorial(s)) * (math.factorial((n+abs(m))/2-s)) * (math.factorial((n-abs(m))/2-s))))


,class,Amplitude,Phase
0,1,0.001243,37.58
1,1,0.001523,37.53
2,1,0.002126,159.09
3,1,0.006693,75.84
4,1,0.000113,-48.87
...,...,...,...
14965,4,0.086193,-178.69
14966,4,0.081538,175.34
14967,4,0.073088,165.08
14968,4,0.071932,154.61


### Compute the Zernike moments for an order and repetition (n, m)
[(n, m) : "(0, 0)", "(1, 1)", "(2, 0)", "(2, 2)", "(3, 1)",
        "(3, 3)", "(4, 0)", "(4, 2)", "(4, 4)", "(5, 1)", "(5, 3)",
        "(5, 5)", "(6, 0)", "(6, 2)", "(6, 4)", "(6, 6)" ,"(7, 1)",
        "(7, 3)", "(7, 5)", "(7, 7)"]

In [4]:
m = []
n = []
N = np.arange(0 , 8)
M = np.arange(0 , 8)
for i in range (0,8):
    for j in range (0,8):
        if(((N[i] - abs(M[j])) % 2 == 0 ) and (abs(M[j]) <= N[i])): # n - |m| = pair et |m| <= n
            n.append(N[i])
            m.append(M[j])
            
classes = {
	"circle"   : 1,
	"square"   : 2,
	"star"     : 3,
	"triangle" : 4
}
zer_m = []
img_path = "archive/shapes/"
csv_file ="archive/shapes/Zernik_dataset_ordre20.csv"
for c, idx in classes.items():
    class_folder = img_path + "{}/".format(c)
    for f in os.listdir(class_folder):
        fpath = class_folder + f
        sample = int(f.replace(".png", ""))
        img = cv.imread(fpath, cv.IMREAD_GRAYSCALE)
        img_ = np.logical_not(img)
        img_ = ~img_
        A_Z_m = []
        for i in range (0,20):
            [A, Phi] = Zernikmoment(img_,n[i],m[i])    # Call Zernikemoment fuction
            A_Z_m.append(round(A,4))
        A_Z_m = np.array(A_Z_m)
        A_Z_m = [idx] + A_Z_m.reshape((1, 20)).tolist()[0]
        zer_m.append(A_Z_m)
        #cols = ["class", "A0","A1","A2","A3","A4","A5","A6","A7","A8","A9","A10","A11","A12","A13","A14","A15","A16","A17","A18","A19"]

cols = ["class", "(0, 0)", "(1, 1)", "(2, 0)", "(2, 2)", "(3, 1)",
        "(3, 3)", "(4, 0)", "(4, 2)", "(4, 4)", "(5, 1)", "(5, 3)",
        "(5, 5)", "(6, 0)", "(6, 2)", "(6, 4)", "(6, 6)" ,"(7, 1)",
        "(7, 3)", "(7, 5)", "(7, 7)"]

df = pd.DataFrame(zer_m, columns=cols)
df.to_csv(csv_file, index=None)
print("Feature Extraction finish!")
df

C:\Users\HP\AppData\Local\Temp/ipykernel_7256/3140508661.py:5: DeprecationWarning: Using factorial() with floats is deprecated
  c = (-1)**(s) * ( math.factorial(n-s) / ((math.factorial(s)) * (math.factorial((n+abs(m))/2-s)) * (math.factorial((n-abs(m))/2-s))))


Feature Extraction finish!


,class,"(0, 0)","(1, 1)","(2, 0)","(2, 2)","(3, 1)","(3, 3)","(4, 0)","(4, 2)","(4, 4)",...,"(5, 3)","(5, 5)","(6, 0)","(6, 2)","(6, 4)","(6, 6)","(7, 1)","(7, 3)","(7, 5)","(7, 7)"
0,1,1.0039,0.0009,0.2277,0.0027,0.0022,0.0002,0.9120,0.0085,0.0008,...,0.0008,0.0001,1.8833,0.0106,0.0014,0.0001,0.0017,0.0014,0.0004,0.0001
1,1,1.0037,0.0011,0.2274,0.0028,0.0026,0.0003,0.9122,0.0089,0.0008,...,0.0013,0.0001,1.8840,0.0110,0.0014,0.0001,0.0020,0.0022,0.0003,0.0001
2,1,1.0038,0.0014,0.2274,0.0026,0.0033,0.0004,0.9121,0.0082,0.0007,...,0.0015,0.0002,1.8838,0.0103,0.0007,0.0001,0.0022,0.0027,0.0010,0.0001
3,1,1.0043,0.0044,0.2283,0.0018,0.0106,0.0007,0.9117,0.0057,0.0009,...,0.0026,0.0005,1.8821,0.0071,0.0017,0.0001,0.0072,0.0044,0.0026,0.0000
4,1,0.9149,0.0032,0.1290,0.0122,0.0060,0.0010,1.0408,0.0317,0.0016,...,0.0034,0.0003,2.1441,0.0210,0.0042,0.0006,0.0083,0.0036,0.0014,0.0003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14965,4,1.0707,0.0449,0.3744,0.0132,0.1063,0.0289,0.9856,0.0354,0.0156,...,0.1002,0.0074,1.7398,0.0606,0.0583,0.0072,0.0066,0.1507,0.0280,0.0059
14966,4,1.0680,0.0445,0.3704,0.0135,0.1037,0.0309,0.9851,0.0321,0.0160,...,0.1060,0.0076,1.7425,0.0551,0.0585,0.0082,0.0034,0.1576,0.0269,0.0063
14967,4,1.0677,0.0406,0.3669,0.0129,0.0937,0.0313,0.9781,0.0254,0.0145,...,0.1091,0.0070,1.7389,0.0401,0.0533,0.0084,0.0053,0.1653,0.0233,0.0058
14968,4,1.0702,0.0394,0.3707,0.0127,0.0907,0.0312,0.9771,0.0210,0.0141,...,0.1090,0.0070,1.7337,0.0299,0.0513,0.0087,0.0118,0.1661,0.0217,0.0057
